# Tag 2: Bewegungssteuerung & Laufverhalten
## Unitree G1 Humanoid Roboter Academy

### Agenda Tag 2:
1. Dynamisches Gehen & Stabilität
2. Regelungstechnik für Humanoide
3. Reinforcement Learning Ansätze
4. Keyboard-Tele-Operation (Workshop)
5. Trajektorien-Planung
6. Praktische Aufgaben

---
## 1. Dynamisches Gehen bei Humanoiden

### 1.1 Herausforderungen beim bipedalen Gehen

Im Gegensatz zu Radrobotern oder Quadrupeds müssen Humanoide:

**1. Dynamische Balance halten:**
```
       ┌─────┐
       │ CoM │  ← Center of Mass (Schwerpunkt)
       └──┬──┘
          │
    ┌─────┴─────┐
    │           │
   ┌┴┐         ┌┴┐
   │L│         │R│  ← Support Polygon
   └─┘         └─┘
```

**Bedingung für Stabilität:**
- Center of Mass (CoM) muss über dem Support Polygon bleiben
- Bei Einbeinstand: sehr kleines Polygon!
- Bei Zweibeinstand: größeres Polygon

**2. Schwungphasen koordinieren:**
- Swing Phase: Bein in der Luft
- Stance Phase: Bein trägt Gewicht
- Doppelstützphase: Beide Füße am Boden

**3. Dynamische Kräfte kompensieren:**
- Bodenreaktionskräfte
- Trägheitskräfte während Beschleunigung
- Störungen (Unebenheiten, Schubser)

### 1.2 Zero Moment Point (ZMP) Stabilität

**ZMP-Konzept:**
- Punkt am Boden, an dem die Summe aller Momente = 0
- Wenn ZMP innerhalb des Support Polygon → stabil
- Wenn ZMP außerhalb → Roboter kippt

**Mathematische Bedingung:**
```
Σ τ_ZMP = 0
→ Roboter passt CoM-Trajektorie an, um ZMP im Polygon zu halten
```

**Beispiel: Vorwärtsgehen**
```
1. CoM verschiebt sich über rechten Fuß
2. Linker Fuß hebt ab (Swing)
3. Linker Fuß setzt vor rechtem auf
4. CoM verschiebt sich über linken Fuß
5. Rechter Fuß hebt ab...
```

---
## 2. Regelungstechnik im G1

### 2.1 Hierarchische Regelungsarchitektur

```
┌──────────────────────────────────────────┐
│ High-Level: Bewegungsplanung            │
│ Input: Zielgeschwindigkeit (vx, vy, ω)  │
│ Output: Fußplatzierungs-Trajektorien    │
└────────────────┬─────────────────────────┘
                 │
┌────────────────▼─────────────────────────┐
│ Mid-Level: Whole-Body Controller        │
│ - Inverse Kinematik                      │
│ - CoM-Trajektorien                       │
│ - ZMP-Stabilisierung                     │
│ Output: Gelenkwinkel-Sollwerte          │
└────────────────┬─────────────────────────┘
                 │
┌────────────────▼─────────────────────────┐
│ Low-Level: Motor Controller              │
│ - PID-Regelung pro Gelenk               │
│ - Drehmomenten-Kontrolle                │
│ Output: Motor-Kommandos                  │
└──────────────────────────────────────────┘
```

### 2.2 PID-Regelung auf Gelenkebene

Jedes Gelenk hat einen **PID-Controller:**

```python
# Vereinfachtes Modell:
def pid_control(q_target, q_current, dq_current, Kp, Kd, Ki):
    """
    q_target: Ziel-Gelenkwinkel
    q_current: Aktueller Gelenkwinkel
    dq_current: Aktuelle Gelenkgeschwindigkeit
    """
    error = q_target - q_current
    
    # Proportional term
    P = Kp * error
    
    # Derivative term (Dämpfung)
    D = Kd * (-dq_current)
    
    # Integral term (steady-state error)
    I = Ki * integral_error
    
    tau = P + D + I  # Gesamt-Drehmoment
    return tau
```

**Typische G1-Parameter:**
- Beine: Kp=200, Kd=5 (steif für Gewichtsträger)
- Arme: Kp=100, Kd=3 (weicher für Manipulation)
- Hände: Kp=20, Kd=1 (sehr weich für Kontakt)

---
## 3. Reinforcement Learning für Laufverhalten

### 3.1 RL-Ansatz für Humanoide

**Problem:** Klassische Regelung (ZMP, Inverse Kinematik) erfordert:
- Genaue Modelle
- Hand-tuning vieler Parameter
- Schwierig für komplexes Terrain

**Lösung: End-to-End RL**
- Lerne Policy direkt aus Sensordaten → Motor-Kommandos
- Training in Simulation (MuJoCo)
- Transfer zu realem Roboter (Sim-to-Real)

### 3.2 G1 Arm Reach RL-Environment

Das Repository enthält ein **vollständiges RL-Setup** für Arm-Steuerung:

In [ ]:
# Setup für RL-Experimente
import sys
sys.path.append('../unitree_g1_vibes/RL-shenanigans')

# Importiere das Gymnasium Environment
# from g1_arm_rl_env import G1ArmReachEnv

print("RL Environment für G1 Arm-Steuerung verfügbar!")

### 3.3 RL Environment Struktur

**Observation Space (24-dim):**
```python
obs = [
    q[0:7],           # Aktuelle Gelenkwinkel (7 DOF)
    dq[0:7],          # Gelenkgeschwindigkeiten (7 DOF)
    p_hand[0:3],      # Hand-Position (x, y, z)
    p_goal[0:3],      # Ziel-Position (x, y, z)
    delta_p[0:3],     # Vektor Hand→Ziel
    step_fraction     # Fortschritt im Episode (0-1)
]
```

**Action Space (7-dim):**
```python
action = Δq[0:7]  # Inkrementelle Gelenkwinkel-Änderungen
                  # Jede Aktion: -0.05 bis +0.05 rad
```

**Reward Function:**
```python
reward = -5.0 * ||p_hand - p_goal||      # Distanz zum Ziel
         - 0.1 * ||Δq||                   # Aktion-Penalty (smooth motion)
         + 1.0 if goal_reached            # Bonus bei Erfolg
         - 1.0 if self_collision          # Penalty bei Kollision
         - 0.5 if joint_limit_exceeded    # Penalty bei Gelenkgrenzen
```

### 3.4 Training Algorithms

Das Setup unterstützt **3 Algorithmen** (Stable-Baselines3):

| Algorithmus | Typ | Vorteile | Nachteile |
|-------------|-----|----------|----------|
| **PPO** | On-Policy | Stabil, einfach zu tunen | Sample-ineffizient |
| **SAC** | Off-Policy | Sample-effizient, robust | Komplexer, mehr Hyperparameter |
| **TD3** | Off-Policy | Deterministisch, schnell | Weniger stabil als SAC |

**Typisches Training:**
```bash
cd ../unitree_g1_vibes/RL-shenanigans
python3 train_g1_arm_policy.py \
    --algo ppo \
    --total-steps 2_000_000 \
    --num-envs 16 \
    --right-arm
```

**Output:**
- Checkpoints: `models/ppo_g1_right_XXXXXX.zip`
- TensorBoard logs: `runs/g1_arm_reach/`

---
## 4. Workshop: Keyboard Tele-Operation

### 4.1 Keyboard Controller Übersicht

Der `keyboard_controller.py` ist ein **vollständiges Teleop-System**:

**Features:**
- WASD-Steuerung (Vorwärts/Rückwärts/Links/Rechts)
- QE-Rotation (Yaw links/rechts)
- Space: Sofort-Stopp
- Shift: Turbo-Modus (höhere Geschwindigkeit)
- Esc: Not-Aus (Damp-Modus)
- Kontinuierliches Velocity-Update (10 Hz)

### 4.2 Starten des Controllers

In [ ]:
# In einem separaten Terminal ausführen:
# cd ../unitree_g1_vibes
# python3 keyboard_controller.py --iface enp68s0f1

print("Keyboard Controller Anleitung:")
print("================================")
print("W/S    : Vorwärts/Rückwärts")
print("A/D    : Rotation Links/Rechts")
print("Q/E    : Seitwärts Links/Rechts")
print("Space  : Sofort Stoppen")
print("Shift  : Turbo-Modus (halten)")
print("Z      : Damp & Beenden")
print("Esc    : Not-Aus")
print("================================")

### 4.3 Internes Funktionsprinzip

```python
# Vereinfachte Version des Controllers:
from pynput import keyboard
import threading

class KeyboardController:
    def __init__(self, bot):
        self.bot = bot
        self.vx = 0.0
        self.vy = 0.0
        self.omega = 0.0
        self.active_keys = set()
        
    def on_press(self, key):
        if key == keyboard.Key.w:
            self.active_keys.add('w')
        elif key == keyboard.Key.s:
            self.active_keys.add('s')
        # ... weitere Keys
        
    def on_release(self, key):
        if 'w' in str(key):
            self.active_keys.discard('w')
        # ... weitere Keys
    
    def update_loop(self):
        """Läuft mit 10 Hz"""
        while self.running:
            # Berechne Geschwindigkeiten aus active_keys
            self.compute_velocities()
            
            # Sende an Roboter
            self.bot.Move(self.vx, self.vy, self.omega, 
                         continuous_move=True)
            
            time.sleep(0.1)  # 10 Hz
```

### 4.4 Geschwindigkeits-Ramping

**Problem:** Abrupte Geschwindigkeitsänderungen → instabil

**Lösung:** Smooth Acceleration/Deceleration

```python
def smooth_velocity_update(current_vel, target_vel, accel_limit, dt):
    """
    Begrenzt Beschleunigung auf accel_limit m/s²
    """
    delta_v = target_vel - current_vel
    max_delta = accel_limit * dt  # z.B. 1.0 m/s² * 0.1s = 0.1 m/s
    
    if abs(delta_v) > max_delta:
        delta_v = max_delta * np.sign(delta_v)
    
    return current_vel + delta_v

# Beispiel:
# current_vx = 0.0
# target_vx = 0.6  (W gedrückt)
# 
# Nach 0.1s: vx = 0.1
# Nach 0.2s: vx = 0.2
# ...
# Nach 0.6s: vx = 0.6 (erreicht)
```

---
## 5. Trajektorien-Planung

### 5.1 Warum Trajektorien?

**Problem mit direkter Geschwindigkeitssteuerung:**
- Keine Garantie für smooth motion
- Keine Kollisionsvermeidung
- Keine Optimierung (z.B. minimale Energie)

**Trajektorien-Planung:**
- Plant gesamten Pfad von Start zu Ziel
- Berücksichtigt Kinematik & Dynamik
- Optimiert nach Kriterien (Zeit, Energie, Glattheit)

### 5.2 Einfache Trajektorien: Geradlinige Bewegung

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def linear_trajectory(start, goal, duration, dt=0.1):
    """
    Erzeugt geradlinige Trajektorie von start zu goal
    
    start, goal: [x, y] Positionen
    duration: Zeit in Sekunden
    dt: Zeitschritt
    """
    num_steps = int(duration / dt)
    trajectory = []
    
    for i in range(num_steps + 1):
        alpha = i / num_steps  # 0 bis 1
        pos = start + alpha * (goal - start)
        trajectory.append(pos)
    
    return np.array(trajectory)

# Beispiel:
start = np.array([0.0, 0.0])
goal = np.array([2.0, 1.0])
traj = linear_trajectory(start, goal, duration=5.0)

plt.figure(figsize=(8, 6))
plt.plot(traj[:, 0], traj[:, 1], 'b-o', label='Trajektorie')
plt.plot(start[0], start[1], 'go', markersize=10, label='Start')
plt.plot(goal[0], goal[1], 'ro', markersize=10, label='Ziel')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title('Geradlinige Trajektorie')
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()

print(f"Trajektorie hat {len(traj)} Wegpunkte")

### 5.3 Geschwindigkeitsprofile

**Trapezoidal Velocity Profile:**
- Beschleunigungsphase (konstant a)
- Konstante Geschwindigkeit
- Abbremsphase (konstant -a)

In [ ]:
def trapezoidal_velocity_profile(distance, v_max, a_max, dt=0.1):
    """
    Erzeugt trapezförmiges Geschwindigkeitsprofil
    
    distance: Gesamtstrecke [m]
    v_max: Maximale Geschwindigkeit [m/s]
    a_max: Maximale Beschleunigung [m/s²]
    """
    # Zeit für Beschleunigung
    t_accel = v_max / a_max
    
    # Strecke während Beschleunigung
    s_accel = 0.5 * a_max * t_accel**2
    
    # Strecke bei konstanter Geschwindigkeit
    s_const = distance - 2 * s_accel
    
    if s_const < 0:
        # Zu kurze Strecke, erreicht v_max nicht
        v_max = np.sqrt(a_max * distance)
        t_accel = v_max / a_max
        s_accel = distance / 2
        s_const = 0
    
    t_const = s_const / v_max if s_const > 0 else 0
    
    # Generiere Profil
    time = []
    velocity = []
    position = []
    
    t = 0
    s = 0
    v = 0
    
    # Phase 1: Beschleunigung
    while t < t_accel:
        time.append(t)
        v = a_max * t
        velocity.append(v)
        s = 0.5 * a_max * t**2
        position.append(s)
        t += dt
    
    # Phase 2: Konstante Geschwindigkeit
    t_const_end = t + t_const
    s_const_start = s
    while t < t_const_end:
        time.append(t)
        velocity.append(v_max)
        s = s_const_start + v_max * (t - t_accel)
        position.append(s)
        t += dt
    
    # Phase 3: Abbremsen
    t_decel_start = t
    s_decel_start = s
    while s < distance:
        time.append(t)
        v = v_max - a_max * (t - t_decel_start)
        if v < 0:
            v = 0
        velocity.append(v)
        s = s_decel_start + v_max * (t - t_decel_start) - 0.5 * a_max * (t - t_decel_start)**2
        position.append(s)
        t += dt
    
    return np.array(time), np.array(velocity), np.array(position)

# Beispiel:
t, v, s = trapezoidal_velocity_profile(distance=5.0, v_max=0.6, a_max=0.5)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

ax1.plot(t, v, 'b-', linewidth=2)
ax1.set_xlabel('Zeit [s]')
ax1.set_ylabel('Geschwindigkeit [m/s]')
ax1.set_title('Trapezförmiges Geschwindigkeitsprofil')
ax1.grid(True)

ax2.plot(t, s, 'r-', linewidth=2)
ax2.set_xlabel('Zeit [s]')
ax2.set_ylabel('Position [m]')
ax2.set_title('Resultierende Position')
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f"Gesamtzeit: {t[-1]:.2f} s")
print(f"Endposition: {s[-1]:.2f} m")

---
## 6. Praktische Aufgaben

### Aufgabe 1: Quadrat fahren

**Ziel:** Programmieren Sie eine Sequenz, die den Roboter ein 2m×2m Quadrat abfahren lässt.

**Hinweise:**
- Nutzen Sie `bot.Move(vx, vy, omega)`
- Planen Sie: Vorwärts → Rotation 90° → Vorwärts → ...
- Berechnen Sie benötigte Zeit für jede Phase

In [ ]:
# Lösung Aufgabe 1:
import time

def drive_square(bot, side_length=2.0, velocity=0.2):
    """
    Fährt ein Quadrat mit gegebener Seitenlänge
    
    bot: LocoClient Instanz
    side_length: Seitenlänge in Metern
    velocity: Fahrgeschwindigkeit in m/s
    """
    # Berechne Zeit für eine Seite
    t_forward = side_length / velocity
    
    # Berechne Zeit für 90° Rotation
    # 90° = π/2 rad, bei omega=0.3 rad/s
    omega = 0.3
    t_rotate = (np.pi / 2) / omega
    
    print(f"Quadrat fahren: {side_length}m Seitenlänge")
    print(f"Zeit pro Seite: {t_forward:.1f}s")
    print(f"Zeit pro Drehung: {t_rotate:.1f}s\n")
    
    for i in range(4):
        print(f"Seite {i+1}/4: Vorwärts fahren...")
        # Vorwärts
        # bot.Move(vx=velocity, vy=0, omega=0)
        # time.sleep(t_forward)
        
        print(f"Seite {i+1}/4: Drehe 90° links...")
        # Rotation
        # bot.Move(vx=0, vy=0, omega=omega)
        # time.sleep(t_rotate)
    
    print("Quadrat abgeschlossen! Stoppe...")
    # bot.StopMove()

# Testen (nur mit verbundenem Roboter!):
# drive_square(bot, side_length=2.0, velocity=0.2)

print("Funktion drive_square() definiert.")
print("Unkommentieren zum Ausführen mit echtem Roboter!")

### Aufgabe 2: Geschwindigkeitsramping implementieren

**Ziel:** Implementieren Sie eine Funktion, die Geschwindigkeiten smooth interpoliert.

**Anforderungen:**
- Beschleunigung begrenzen auf max. 1.0 m/s²
- Von v=0 auf v=0.6 m/s in ~0.6 Sekunden
- Plotten Sie das Geschwindigkeitsprofil

In [ ]:
# Lösung Aufgabe 2:
def smooth_velocity_ramp(v_start, v_target, a_max=1.0, dt=0.1):
    """
    Erzeugt smooth velocity ramp
    """
    velocities = [v_start]
    v_current = v_start
    
    while abs(v_target - v_current) > 0.01:
        delta_v = v_target - v_current
        max_delta = a_max * dt
        
        if abs(delta_v) > max_delta:
            delta_v = max_delta * np.sign(delta_v)
        
        v_current += delta_v
        velocities.append(v_current)
    
    return np.array(velocities)

# Test:
v_profile = smooth_velocity_ramp(v_start=0.0, v_target=0.6, a_max=1.0, dt=0.1)
time_steps = np.arange(len(v_profile)) * 0.1

plt.figure(figsize=(10, 6))
plt.plot(time_steps, v_profile, 'b-o', linewidth=2, markersize=4)
plt.xlabel('Zeit [s]')
plt.ylabel('Geschwindigkeit [m/s]')
plt.title('Smooth Velocity Ramping (a_max = 1.0 m/s²)')
plt.grid(True)
plt.axhline(y=0.6, color='r', linestyle='--', label='Zielgeschwindigkeit')
plt.legend()
plt.show()

print(f"Beschleunigungszeit: {time_steps[-1]:.2f} s")
print(f"Anzahl Steps: {len(v_profile)}")

### Aufgabe 3: Figure-8 Trajektorie

**Ziel:** Planen Sie eine "Achter"-Trajektorie (∞-Form).

**Mathematik:**
```
x(t) = A * sin(ωt)
y(t) = A * sin(2ωt) / 2
```

In [ ]:
# Lösung Aufgabe 3:
def figure_eight_trajectory(amplitude=1.0, period=20.0, dt=0.1):
    """
    Erzeugt Figure-8 Trajektorie
    
    amplitude: Größe der Acht [m]
    period: Umlaufzeit [s]
    dt: Zeitschritt
    """
    omega = 2 * np.pi / period
    t = np.arange(0, period, dt)
    
    x = amplitude * np.sin(omega * t)
    y = amplitude * np.sin(2 * omega * t) / 2
    
    return x, y, t

# Visualisieren:
x, y, t = figure_eight_trajectory(amplitude=1.5, period=30.0)

plt.figure(figsize=(10, 6))
plt.plot(x, y, 'b-', linewidth=2)
plt.plot(x[0], y[0], 'go', markersize=10, label='Start')
plt.plot(x[-1], y[-1], 'ro', markersize=10, label='Ende')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title('Figure-8 Trajektorie')
plt.grid(True)
plt.axis('equal')
plt.legend()
plt.show()

print(f"Trajektorie hat {len(x)} Wegpunkte")
print(f"Gesamtdauer: {t[-1]:.1f} s")

---
## 7. Zusammenfassung Tag 2

### Gelernte Konzepte:
✓ Dynamisches Gehen & ZMP-Stabilität  
✓ Hierarchische Regelungsarchitektur  
✓ PID-Kontrolle auf Gelenkebene  
✓ Reinforcement Learning für Manipulation  
✓ Keyboard-Tele-Operation  
✓ Trajektorien-Planung & Geschwindigkeitsprofile  

### Wichtige Dateien:
- `keyboard_controller.py` - Vollständiges Teleop-System
- `g1_arm_rl_env.py` - RL Environment
- `train_g1_arm_policy.py` - RL Training Pipeline
- `g1_arm_policy_controller.py` - Policy Evaluation

---
## 8. Hausaufgabe / Vertiefung

**Aufgabe 1:** Testen Sie den Keyboard Controller mit echtem Roboter:
- Erforschen Sie verschiedene Geschwindigkeitskombinationen
- Testen Sie die Turbo-Funktion
- Dokumentieren Sie maximale sichere Geschwindigkeiten

**Aufgabe 2:** RL Training (optional):
```bash
cd ../unitree_g1_vibes/RL-shenanigans
python3 train_g1_arm_policy.py --algo ppo --total-steps 100000
```
- Beobachten Sie Training in TensorBoard
- Analysieren Sie Reward-Kurven

**Aufgabe 3:** Erweiterte Trajektorien:
- Implementieren Sie eine Spirale
- Kombinieren Sie Vorwärts + Rotation
- Visualisieren Sie die Trajektorie

**Bonusaufgabe:** Implementieren Sie einen **Joystick-Controller** analog zum Keyboard-Controller.

---
## 9. Nächster Tag: Navigation & Umgebungserkennung

**Vorschau Tag 3:**
- SLAM (Simultaneous Localization and Mapping)
- LiDAR-Integration (Livox MID-360)
- Globale Pfadplanung (A*, RRT)
- Lokale Hindernisvermeidung
- Workshop: Navigationsaufgabe mit Hindernisfeld